In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import jax
from jax import numpy as jnp
import optax
from functools import partial
import sim
import optuna
import matplotlib.pyplot as plt

In [2]:
jax.devices()

[cuda(id=0)]

In [3]:
!optuna-dashboard sqlite:///qlearning4x4.db

Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [08/Feb/2024 09:29:58] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [08/Feb/2024 09:29:58] "GET /dashboard HTTP/1.1" 304 0
127.0.0.1 - - [08/Feb/2024 09:29:59] "GET /static/bundle.js HTTP/1.1" 304 0
127.0.0.1 - - [08/Feb/2024 09:30:00] "GET /api/studies HTTP/1.1" 200 137
127.0.0.1 - - [08/Feb/2024 09:30:02] "GET /api/meta HTTP/1.1" 200 32
/home/davide_sartor/.conda/envs/dl_env/lib/python3.10/site-packages/optuna/study/_study_summary.py:115: FutureWarning: `system_attrs` has been deprecated in v3.1.0. The removal of this feature is currently scheduled for v5.0.0, but this schedule is subject to change. See https://github.com/optuna/optuna/releases/tag/v3.1.0.
  warnings.warn(
127.0.0.1 - - [08/Feb/2024 09:30:02] "GET /api/studies/1?after=0 HTTP/1.1" 200 88075
127.0.0.1 - - [08/Feb/2024 09:30:02] "GET /api/studies/1/param_importances HTTP/1.1" 200 205
127.0.0.1 - - [08/Feb/2024 09:30:03] "GET /api/studies/1/param_importances 

In [3]:
def objective(trial):
    seed = 42
    map_size = 4
    max_interactions = 100_000
    map_scale = int(jnp.log2(map_size))

    # hyperparams
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    rollout_length = 2 * map_size**2
    n_rollouts = max_interactions // rollout_length
    train_iter = 8

    # setup annealing schedules
    dur = int(train_iter * n_rollouts * trial.suggest_float(f"annealing", 0.1, 1.0 / map_scale))
    annealing_fn = optax.linear_schedule(1.0, 0.01, dur)

    run_simulation = partial(
        sim.run_q_learning_simulation,
        map_size=map_size,
        n_rollouts=n_rollouts,
        rollout_length=rollout_length,
        train_iter=train_iter,
        lr=lr,
        annealing_fn=annealing_fn,
    )
    rng_key = jax.random.PRNGKey(seed)
    env, dql_state, results = jax.vmap(run_simulation)(jax.random.split(rng_key, 8))

    best = results.eval_reward.max(axis=0)
    worst = results.eval_reward.min(axis=0)
    accuracy = (best.mean() + worst.mean())/2

    return accuracy


study = optuna.create_study(
    study_name="qlearning",
    storage="sqlite:///qlearning4x4.db",
    load_if_exists=True,
    direction="maximize"
)

study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-02-08 07:36:25,511] A new study created in RDB with name: qlearning


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

[I 2024-02-08 07:37:16,137] Trial 0 finished with value: 0.11687999963760376 and parameters: {'lr': 0.0041823181213999, 'annealing': 0.42434748874478134}. Best is trial 0 with value: 0.11687999963760376.


  0%|          | 0/3125 [00:00<?, ?it/s]

...

[I 2024-02-08 08:43:42,997] Trial 97 finished with value: 0.13224999606609344 and parameters: {'lr': 0.00024844402521666096, 'annealing': 0.4446560686823643}. Best is trial 91 with value: 0.13518500328063965.


  0%|          | 0/3125 [00:00<?, ?it/s]

[I 2024-02-08 08:44:27,316] Trial 98 finished with value: 0.13223999738693237 and parameters: {'lr': 0.0007291619622487387, 'annealing': 0.4836842067060735}. Best is trial 91 with value: 0.13518500328063965.


  0%|          | 0/3125 [00:00<?, ?it/s]

[I 2024-02-08 08:45:08,701] Trial 99 finished with value: 0.13352499902248383 and parameters: {'lr': 0.00029707973368550477, 'annealing': 0.4403960523962793}. Best is trial 91 with value: 0.13518500328063965.
